In [74]:
# Import Libraries

import os
import pandas as pd
import requests as req
from splinter import Browser
from bs4 import BeautifulSoup as bs
import time
# from selenium import webdriver  
from lxml import html

## Getting news from https://mars.nasa.gov/news/ web page

In [79]:
# Chromedriver execution

executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=True)

# URL path
url1="https://mars.nasa.gov/news/"
browser.visit(url1)
# Save html and parser
html=browser.html
soup = bs(html, "html.parser")

In [80]:
# Printing
news_date = soup.find('li', class_='slide').find('div', class_="list_date").text
print(news_date)
news_title = soup.find('div',class_="list_text").find('div', class_="content_title").text
print(news_title)
news_text = soup.find('div',class_="list_text").find('div',class_="article_teaser_body").text
print(news_text)

October 27, 2020
NASA's Perseverance Rover Is Midway to Mars 
Sometimes half measures can be a good thing – especially on a journey this long. The agency's latest rover only has about 146 million miles left to reach its destination.


## Getting featured image from https://www.jpl.nasa.gov/spaceimages/ web page

In [81]:
# URL path
url2 = "https://www.jpl.nasa.gov/spaceimages/"
# Visiting url2 to click and response
browser.visit(url2)
browser.find_by_id('full_image').click()
time.sleep(3)

# Clicking on more info button
browser.links.find_by_partial_text('more info').click()

# Getting mage URL
featured_image_url = browser.find_by_xpath("//img[@class='main_image']")._element.get_attribute("src")

print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA16715_hires.jpg
